# Homework for Week 3

In [1]:
import pandas as pd
import numpy as np

In [2]:
d = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
!wget $d

--2025-10-14 01:42:18--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.1’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.1s    

2025-10-14 01:42:18 (766 KB/s) - ‘course_lead_scoring.csv.1’ saved [80876/80876]



In [5]:
df = pd.read_csv("course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [49]:
categorical = ["lead_source", "industry", "employment_status", "location" ]
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score" ]

df[categorical] = df[categorical].fillna("NA")
df[numerical] = df[numerical].fillna(0)
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [50]:
# Q1 Most frequent observation in industry
df.industry.mode(), df.industry.value_counts()

(0    retail
 Name: industry, dtype: object,
 industry
 retail           203
 finance          200
 other            198
 healthcare       187
 education        187
 technology       179
 manufacturing    174
 NA               134
 Name: count, dtype: int64)

In [51]:
# Q2 Biggest correlation coefficient 

il_corr = df['interaction_count'].corr(df['lead_score'])
nl_corr = df['number_of_courses_viewed'].corr(df['lead_score'])
ni_corr = df['number_of_courses_viewed'].corr(df['interaction_count'])
ai_corr = df['annual_income'].corr(df['interaction_count'])

il_corr, nl_corr, ni_corr, ai_corr


(np.float64(0.009888182496913084),
 np.float64(-0.004878998354681256),
 np.float64(-0.023565222882888044),
 np.float64(0.027036472404814348))

In [52]:
from sklearn.model_selection import train_test_split


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [53]:
df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [54]:
# Q3 - mi between categorical and converted
from sklearn.metrics import mutual_info_score

mi = {}
for c in categorical:
    mi[c] = mutual_info_score(df_full_train[c], df_full_train.converted)

round(pd.Series(mi).sort_values(ascending=False), 2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [73]:
# Q4 Training Logistic regression model
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

accuracy = round(model.score(X_val, y_val), 2)
accuracy

0.7

In [77]:
# Q5 - Finding the least useful feature (categorical only)

original_accuracy = model.score(X_val, y_val)

features_to_exclude = ['industry', 'employment_status', 'lead_score']
accuracy_differences = {}

for f in features_to_exclude:
    # Createing list of all features except the current one
    features_subset = [feat for feat in (categorical + numerical) if feat != f]
    
    # Preparing the training data without this feature
    train_dict_subset = df_train[features_subset].to_dict(orient='records')
    dv_temp = DictVectorizer(sparse=False)
    X_train_subset = dv_temp.fit_transform(train_dict_subset)
    
    # Preparing validation data without this feature
    val_dict_subset = df_val[features_subset].to_dict(orient='records')
    X_val_subset = dv_temp.transform(val_dict_subset)
    
    # Training the model without this feature
    model_temp = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model_temp.fit(X_train_subset, y_train)
    
    # Calculating accuracy
    accuracy_subset = model_temp.score(X_val_subset, y_val)
    
    # Storing the difference
    accuracy_differences[f] = original_accuracy - accuracy_subset

results = pd.Series(accuracy_differences).sort_values()
results

lead_score          -0.006826
industry             0.000000
employment_status    0.003413
dtype: float64

In [92]:
# Q6 - Training a regulaized LG

c_values = [0.01, 0.1, 1, 10, 100]

for c in c_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    print(c, ": ", round(model.score(X_val, y_val), 3))



0.01 :  0.7
0.1 :  0.7
1 :  0.7
10 :  0.7
100 :  0.7
